## TF-IDF Vectorizer From Scratch

<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

### Corpus

In [55]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### Implementation From Scratch

In [62]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [63]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [3]:
def get_feature_names(corpus):       #function to get unique list of words from corpus or the vocab

    res = []
    words = []
    unique_word_list = []
    
    for sen in corpus:
        res.append(sen.split())
    
    for doc in res:
        for word in doc:
            words.append(word)

    unique_words = Counter(words)
    
    for key,val in unique_words.items():
        unique_word_list.append(key)
    return sorted(unique_word_list)

#------------------------------------------------------------------------------------------------

def fit(corpus):       #custom fit function that returns corresponding idf values of the vocab

    N = len(corpus)
    res = []
    for sen in corpus:
        res.append(sen.split())
        
    idf_val = []
    count = 0
    unique_word_list = get_feature_names(corpus)
                                     
    for word in unique_word_list:
        for doc in res:
            if word in doc:
                count += 1
        idf_val.append(1+(math.log((1+len(corpus))/(1+count))))  #calculating idf value using sklearn idf formula
        count = 0
    return idf_val

#--------------------------------------------------------------------------------------------------

def transform(corpus):        #custom transform function that returns a sparse matrix representing tfidf values of the corpus
    
    res = []
    idf = fit(corpus)

    unique_word_list = get_feature_names(corpus)
#     ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

    idf_dict = {unique_word_list[i]:idf[i] for i in range(len(unique_word_list))} # a dictionary of vocab and idf values
    #  {'and': 1.916290731874155,
    #  'document': 1.2231435513142097,
    #  'first': 1.5108256237659907,
    #  'is': 1.0,
    #  'one': 1.916290731874155,
    #  'second': 1.916290731874155,
    #  'the': 1.0,
    #  'third': 1.916290731874155,
    #  'this': 1.0}

    for sen in corpus:
        res.append(sen.split())
        
#         res is shown below
#         [['this', 'is', 'the', 'first', 'document'],
#          ['this', 'document', 'is', 'the', 'second', 'document'],
#          ['and', 'this', 'is', 'the', 'third', 'one'],
#          ['is', 'this', 'the', 'first', 'document']]


    mat = [[0 for i in range(len(unique_word_list))] for j in range(len(res))]


    i = 0
    for row in res:

        freq_dic = Counter(row) #Counter({'this': 1, 'is': 1, 'the': 1, 'first': 1, 'document': 1})
        for key,val in freq_dic.items():

            tfidf_val = (idf_dict[key])*(val/len(row)) #calculating tfidf as idf*term-frequency of each term in document
            mat[i][unique_word_list.index(key)] = tfidf_val
        i+=1
        if i == len(res):
            break

    arr = numpy.array(mat)
    arr = csr_matrix(arr)
    normalized_arr = normalize(arr, norm = 'l2')
    return normalized_arr

In [4]:
print(get_feature_names(corpus))

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [5]:
print(fit(corpus))

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [6]:
output = transform(corpus)
output.shape

(4, 9)

In [7]:
print(output[0])

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149


### SkLearn Implementation for verification

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [57]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [58]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [59]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [60]:
# sklearn tfidf values for first line of the above corpus.

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [61]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### THE END